<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=58f0bc35752482b80f5b2a95bd6e078849568c0b4b2ecada912ff183871c9cfd
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-31 11:25:27
-------------------
qualified stocks: 88
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.37 K
Current:  1.43 C
-------------------
Today PnL: 91.92 K (0.65%)
Current PnL: -26.29 L (-16.77%)
CY Booked + Current PnL: -12.00 L (-7.66%)
-------------------
Total profit:  1.46 L
Total loss:  -27.76 L
-------------------
Total Booked + Current PnL: 14.98 L (11.53%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.01%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 91.67 L (64.22%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.00,1.05,-19.55,24.41,0.09,19791.0,-19701.0,81076.0,88.26,40.0,M-SC,3.17,253.0,-1.00,0.57,6.52,OX40N,NTT,DURABLES
53,NESTLEIND,1377.00,0.61,13.01,7.42,21.40,22257.0,34539.0,299965.0,4.80,64.0,X-LC,5.19,12.0,1.55,2.10,20.16,XY25,NTT,FMCG
51,MASFIN,398.61,2.26,-1.55,23.97,22.05,23123.0,-1515.0,96465.0,-15.09,55.0,H-SC,6.58,164.0,-0.07,0.68,40.22,XR,ATH,FINANCE
84,VOLTAS,1530.00,0.35,6.03,12.88,19.69,26186.0,11568.0,203310.0,-4.26,45.0,X-MC,2.12,78.0,0.44,1.42,13.24,XY25,NTT,AC
44,ITC,452.00,0.65,-1.18,12.10,10.78,28736.0,-2833.0,237485.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,4.00,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,5.06,0.20,46.66,46.95,58580.0,249.0,125546.0,-7.26,68.0,H-SC,9.40,154.0,0.00,0.88,57.28,MH,ATH,POWER
4,ALKYLAMINE,4546.37,4.04,-30.97,180.50,93.63,125798.0,-31269.0,69694.0,-26.06,46.0,H-SC,15.71,145.0,-0.25,0.49,4.04,SR,ATH,CHEMICALS
10,BAJAJHLDNG,14451.52,3.76,2.53,26.82,30.03,36674.0,3370.0,136740.0,-0.67,52.0,X-LC,12.14,35.0,0.09,0.96,11.24,X40,ATH,FINANCE
66,SHALBY,327.00,2.78,-22.72,61.88,25.10,86623.0,-41157.0,139986.0,928.71,48.0,M-SC,9.15,250.0,-0.48,0.98,12.18,XY24,NTT,HEALTHCARE
21,DABUR,735.00,2.41,-0.76,44.80,43.70,110291.0,-1882.0,246186.0,-8.87,59.0,X-MC,1.99,72.0,-0.02,1.72,14.18,XY24,BTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,HAPPSTMNDS,1480.71,-3.93,-43.30,226.04,84.85,164250.0,-55501.0,72664.0,-32.68,28.0,H-SC,13.70,147.0,-0.34,0.51,0.00,AR,ATH,IT
57,RAJESHEXPO,518.00,-2.63,-63.88,177.01,0.07,87717.0,-87622.0,49555.0,1715.66,43.0,L-SC,4.40,268.0,-1.00,0.35,23.25,OX40N,NTT,JEWELLERY
19,COFFEEDAY,80.00,-1.60,-37.66,116.98,35.27,82810.0,-42759.0,70790.0,-51.71,55.0,L-SC,7.01,270.0,-0.52,0.50,73.26,XR,NTT,HOTELS
39,INDIAMART,4810.62,-1.56,-4.95,117.49,106.72,137735.0,-6105.0,117231.0,-52.92,41.0,H-SC,5.87,139.0,-0.04,0.82,17.27,AR,ATH,MISC
75,TCS,4389.97,-0.88,-11.51,36.41,20.71,121862.0,-43544.0,334693.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.83,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,5.06,0.2,46.66,46.95,58580.0,249.0,125546.0,-7.26,68.0,H-SC,9.4,154.0,0.0,0.88,57.28,MH,ATH,POWER


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,1.13,-6.26,121.80,107.91,168570.0,-9245.0,138399.0,-23.79,41.0,M-SC,11.64,216.0,-0.05,0.97,3.19,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,1.92,-18.73,23.08,0.02,32740.0,-32703.0,141856.0,116.52,39.0,M-SC,8.37,234.0,-1.00,0.99,23.39,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,1.05,-19.55,24.41,0.09,19791.0,-19701.0,81076.0,88.26,40.0,M-SC,3.17,253.0,-1.00,0.57,6.52,OX40N,NTT,DURABLES
48,KANSAINER,340.00,0.93,-21.23,44.05,13.47,93574.0,-57240.0,212427.0,-67.53,55.0,H-SC,6.49,158.0,-0.61,1.49,8.01,XY24,NTT,PAINTS
68,SIS,528.00,-0.03,-23.41,58.32,21.26,49596.0,-25990.0,85042.0,2007.25,52.0,H-SC,4.48,166.0,-0.52,0.60,14.90,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,5.06,0.20,46.66,46.95,58580.0,249.0,125546.0,-7.26,68.0,H-SC,9.40,154.0,0.00,0.88,57.28,MH,ATH,POWER
86,WIPRO,420.00,-0.36,7.75,59.88,72.27,109327.0,13128.0,182576.0,-7.44,55.0,M-LC,5.22,99.0,0.12,1.28,14.40,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.00,-0.36,7.75,59.88,72.27,109327.0,13128.0,182576.0,-7.44,55.0,M-LC,5.22,99.0,0.12,1.28,14.40,XR,NTT,IT
49,KPIGREEN,730.66,5.06,0.20,46.66,46.95,58580.0,249.0,125546.0,-7.26,68.0,H-SC,9.40,154.0,0.00,0.88,57.28,MH,ATH,POWER
6,ASIANTILES,137.00,-0.01,-1.09,83.67,81.67,77637.0,-1020.0,92790.0,8355.56,66.0,L-SC,16.00,271.0,-0.01,0.65,79.69,XR,NTT,CERAMICS
39,INDIAMART,4810.62,-1.56,-4.95,117.49,106.72,137735.0,-6105.0,117231.0,-52.92,41.0,H-SC,5.87,139.0,-0.04,0.82,17.27,AR,ATH,MISC
26,FINCABLES,1641.55,1.13,-6.26,121.80,107.91,168570.0,-9245.0,138399.0,-23.79,41.0,M-SC,11.64,216.0,-0.05,0.97,3.19,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.14,-19.45,93.00,55.47,182260.0,-47307.0,195979.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.42,X40N,ATH,FINANCE
23,DIXON,18931.72,1.27,-20.65,58.88,26.07,91210.0,-40313.0,154908.0,-61.45,27.0,X-MC,16.13,56.0,-0.44,1.09,1.27,X40N,ATH,IT
35,HONAUT,58357.33,-0.24,-19.16,77.30,43.32,101773.0,-31212.0,131660.0,-30.52,28.0,X-SC,7.23,90.0,-0.31,0.92,1.09,X40N,ATH,ELECTRICAL
28,GLAXO,3466.20,0.26,-4.04,45.01,39.15,86074.0,-8052.0,191232.0,-22.92,29.0,X-MC,10.88,60.0,-0.09,1.34,23.79,X40,ATH,PHARMA
3,ACC,3906.00,-0.16,-27.33,125.87,64.15,217667.0,-65021.0,172930.0,-57.62,33.0,X-SC,5.67,82.0,-0.30,1.21,0.24,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.5,1.04,-17.76,52.61,25.51,80520.0,-33045.0,153050.0,-6.04,40.0,H-LC,0.98,49.0,-0.41,1.07,12.79,AR,ATH,ELECTRICAL
80,UNITDSPR,1644.0,0.11,4.70,15.34,20.77,37606.0,11008.0,245152.0,-4.44,50.0,X-MC,1.52,62.0,0.29,1.72,11.94,X40N,NTT,BREWERIES
13,BERGEPAINT,680.0,1.39,-5.07,27.62,21.14,59605.0,-11535.0,215804.0,-11.79,42.0,X-MC,1.64,74.0,-0.19,1.51,19.84,XY24,NTT,PAINTS
21,DABUR,735.0,2.41,-0.76,44.80,43.70,110291.0,-1882.0,246186.0,-8.87,59.0,X-MC,1.99,72.0,-0.02,1.72,14.18,XY24,BTT,FMCG
44,ITC,452.0,0.65,-1.18,12.10,10.78,28736.0,-2833.0,237485.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,4.00,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.16,-27.33,125.87,64.15,217667.0,-65021.0,172930.0,-57.62,33.0,X-SC,5.67,82.0,-0.30,1.21,0.24,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-0.14,-19.45,93.00,55.47,182260.0,-47307.0,195979.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.42,X40N,ATH,FINANCE
20,COLPAL,3726.84,0.61,-21.55,80.39,41.51,166086.0,-56765.0,206600.0,-6.53,35.0,X-MC,8.40,61.0,-0.34,1.45,0.61,XY25,ATH,FMCG
12,BATAINDIA,2096.00,0.67,-39.09,121.96,35.20,95589.0,-50293.0,78377.0,2.82,36.0,X-SC,17.36,92.0,-0.53,0.55,0.67,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,0.82,-10.99,43.93,28.11,78756.0,-22135.0,179275.0,-35.49,35.0,X-MC,15.20,55.0,-0.28,1.26,0.82,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.66,-29.62,102.97,42.84,47108.0,-19257.0,45749.0,-53.72,44.0,X-SC,31.58,83.0,-0.41,0.32,3.50,XY24,NTT,MISC
60,RELAXO,1176.00,0.89,-47.68,191.13,52.31,145150.0,-69217.0,75943.0,-44.34,45.0,X-SC,7.39,91.0,-0.48,0.53,1.65,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,0.67,-39.09,121.96,35.20,95589.0,-50293.0,78377.0,2.82,36.0,X-SC,17.36,92.0,-0.53,0.55,0.67,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.50,-3.38,28.42,24.08,34858.0,-4297.0,122653.0,-11.26,41.0,X-SC,7.71,89.0,-0.12,0.86,15.57,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,-0.24,-19.16,77.30,43.32,101773.0,-31212.0,131660.0,-30.52,28.0,X-SC,7.23,90.0,-0.31,0.92,1.09,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-0.88,-11.51,36.41,20.71,121862.0,-43544.0,334693.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.83,X40,ATH,IT
42,INFY,1972.00,-0.38,9.97,22.08,34.24,75973.0,31192.0,344080.0,-14.21,52.0,X-LC,7.31,2.0,0.41,2.41,19.08,X40,NTT,IT
77,TMPV,600.00,0.93,-25.17,64.34,22.98,108761.0,-56857.0,169041.0,-21.86,56.0,X-LC,5.86,3.0,-0.52,1.18,6.32,XY24,NTT,AUTO
83,VBL,671.64,0.32,-1.83,38.20,35.67,118446.0,-5774.0,310068.0,-13.42,60.0,X-LC,2.92,4.0,-0.05,2.17,11.77,X40N,ATH,FMCG
44,ITC,452.00,0.65,-1.18,12.10,10.78,28736.0,-2833.0,237485.0,-41.15,49.0,X-LC,1.99,5.0,-0.10,1.66,4.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,1.75,-36.79,110.23,32.89,53796.0,-28404.0,48803.0,-702.61,57.0,L-MC,6.64,259.0,-0.53,0.34,34.40,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.01,-1.09,83.67,81.67,77637.0,-1020.0,92790.0,8355.56,66.0,L-SC,16.00,271.0,-0.01,0.65,79.69,XR,NTT,CERAMICS
51,MASFIN,398.61,2.26,-1.55,23.97,22.05,23123.0,-1515.0,96465.0,-15.09,55.0,H-SC,6.58,164.0,-0.07,0.68,40.22,XR,ATH,FINANCE
14,BSOFT,831.70,1.11,-22.35,94.19,50.79,102871.0,-31437.0,109216.0,-0.27,52.0,H-SC,11.09,151.0,-0.31,0.77,27.16,XR,ATH,IT
49,KPIGREEN,730.66,5.06,0.20,46.66,46.95,58580.0,249.0,125546.0,-7.26,68.0,H-SC,9.40,154.0,0.00,0.88,57.28,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,5.06,0.20,46.66,46.95,58580.0,249.0,125546.0,-7.26,68.0,H-SC,9.40,154.0,0.00,0.88,57.28,MH,ATH,POWER
6,ASIANTILES,137.00,-0.01,-1.09,83.67,81.67,77637.0,-1020.0,92790.0,8355.56,66.0,L-SC,16.00,271.0,-0.01,0.65,79.69,XR,NTT,CERAMICS
14,BSOFT,831.70,1.11,-22.35,94.19,50.79,102871.0,-31437.0,109216.0,-0.27,52.0,H-SC,11.09,151.0,-0.31,0.77,27.16,XR,ATH,IT
76,TITAGARH,1548.00,1.94,-15.69,72.85,45.74,148749.0,-37992.0,204185.0,12.33,65.0,H-SC,4.85,174.0,-0.26,1.43,32.80,XY24,NTT,ENGINEERING
37,ICICIPRULI,790.00,1.44,9.92,18.93,30.73,40992.0,19541.0,216546.0,-15.76,68.0,X-MC,4.27,75.0,0.48,1.52,23.95,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.95
1,25,44.12
2,50,75.09


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.21
MC    30.15
LC    25.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.49
X40      23.49
X40N     12.60
XY25      9.94
XR        9.63
AR        8.62
OX40N     7.35
SR        0.99
MH        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.71
H-SC    23.60
X-LC    22.24
M-SC    11.27
X-SC     7.84
H-MC     4.72
L-SC     1.50
M-MC     1.38
M-LC     1.28
H-LC     1.07
L-LC     1.04
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.10,-6.82,41.17
IT,12.51,-17.77,77.00
FINANCE,10.49,-15.34,64.31
MISC,7.02,-29.69,82.74
ELECTRICAL,5.82,-12.90,54.25
PAINTS,5.70,-13.32,30.16
INSURANCE,4.69,-1.54,36.30
PHARMA,3.96,-4.01,36.76
AUTO,2.81,-30.46,75.69


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3140411.0,21
AR,1341443.0,10
XR,1287658.0,13
X40,1043578.0,15
X40N,885940.0,9
OX40N,742305.0,10
XY25,387391.0,7
SR,279912.0,2
MH,58580.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3612326.0,25
M-SC,1448083.0,15
X-MC,1442279.0,16
X-LC,888861.0,13
X-SC,799334.0,8
H-MC,406107.0,3
L-SC,248164.0,3
M-LC,109327.0,1
H-LC,80520.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1241964.0      6
           AR         917217.0      5
M-SC       XY24       816250.0      6
H-SC       XR         785378.0      7
X-MC       X40        490607.0      7
           XY24       404913.0      3
X-LC       X40        369685.0      6
X-MC       X40N       354487.0      4
H-SC       OX40N      329275.0      4
M-SC       OX40N      325313.0      5
X-SC       X40N       316415.0      3
           XY24       299633.0      3
H-SC       SR         279912.0      2
H-MC       AR         215896.0      2
X-LC       X40N       215038.0      2
X-MC       XY25       192272.0      2
H-MC       XY24       190211.0      1
X-LC       XY24       187440.0      2
X-SC       X40        183286.0      2
M-SC       XR         178710.0      2
L-SC       XR         160447.0      2
M-SC       AR         127810.0      2
X-LC       XY25       116698.0      3
M-LC       XR         109327.0      1
L-SC       OX40N       87717.0      1
H-LC       AR          80520.0      1
H-SC       MH          58580.0      1
L-MC       XR          53796.0      1
M-MC       XY25        47829.0      1
L-LC       XY25        30592.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
